<a href="https://colab.research.google.com/github/chetansolanke14/DeepLearning_Project/blob/master/Garden_Nerd_Flower_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!wget https://he-public-data.s3-ap-southeast-1.amazonaws.com/HE_Challenge_data.zip

--2019-09-22 16:53:18--  https://he-public-data.s3-ap-southeast-1.amazonaws.com/HE_Challenge_data.zip
Resolving he-public-data.s3-ap-southeast-1.amazonaws.com (he-public-data.s3-ap-southeast-1.amazonaws.com)... 52.219.36.191
Connecting to he-public-data.s3-ap-southeast-1.amazonaws.com (he-public-data.s3-ap-southeast-1.amazonaws.com)|52.219.36.191|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 948965311 (905M) [application/zip]
Saving to: ‘HE_Challenge_data.zip’

HE_Challenge_data.z 100%[===================>] 905.00M  10.3MB/s    in 92s     

2019-09-22 16:54:51 (9.83 MB/s) - ‘HE_Challenge_data.zip’ saved [948965311/948965311]



In [0]:
#!unzip HE_Challenge_data.zip

In [0]:
train_csv =pd.read_csv('data/train.csv')
test_csv =pd.read_csv('data/test.csv')
samp_csv =pd.read_csv('data/sample_submission.csv')

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
import cv2 as cv

In [54]:
# We have grayscale images, so while loading the images we will keep grayscale=True, if you have RGB images, you should set grayscale as False
train_image = []
for i in tqdm(range(train_csv.shape[0])):
    img = image.load_img('data/train/'+train_csv['image_id'][i].astype('str')+'.jpg', target_size=(28,28,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

  0%|          | 0/18540 [00:00<?, ?it/s]


TypeError: ignored

In [0]:
y=train_csv['category'].values
y = to_categorical(y)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [28]:
test_image = []
for i in tqdm(range(test_csv.shape[0])):
    img = image.load_img('data/test/'+test_csv['image_id'][i].astype('str')+'.jpg', target_size=(28,28,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

100%|██████████| 2009/2009 [00:10<00:00, 195.71it/s]


In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [0]:
#Data augmentaion process

from keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(width_shift_range=0.1,
                  height_shift_range=0.1,
                  zoom_range=0.2,
                  shear_range=0.1,
                  rotation_range=10)

datagen.fit(X_train)

In [0]:
batches=datagen.flow(X_train,y_train,batch_size=1000)

X_batch,y_batch=next(batches)

In [0]:
def modified_model():
  model=Sequential()
  model.add(Conv2D(60,(5,5),input_shape=(28,28,1),activation='relu'))
  model.add(Conv2D(60,(5,5),activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  #model.add(Dropout(0.5))
  
  model.add(Conv2D(30,(3,3),activation='relu'))
  model.add(Conv2D(30,(3,3),activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  #model.add(Dropout(0.5))
  
  model.add(Flatten())
  model.add(Dense(500,activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(103,activation='softmax'))
  #Compile Model
  model.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
  return model

In [75]:
model =modified_model()
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 24, 24, 60)        1560      
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 20, 20, 60)        90060     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 10, 10, 60)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 8, 8, 30)          16230     
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 6, 6, 30)          8130      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 3, 3, 30)          0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 270)              

In [80]:
history=model.fit_generator(datagen.flow(X_train,y_train,batch_size=50),steps_per_epoch=500,epochs=5,validation_data=(X_test,y_test),shuffle=1)

Epoch 1/5
500/500 [==============================] - 222s 443ms/step - loss: 3.7154 - acc: 0.1344 - val_loss: 3.4603 - val_acc: 0.1810
Epoch 2/5
500/500 [==============================] - 234s 468ms/step - loss: 3.5487 - acc: 0.1625 - val_loss: 3.3772 - val_acc: 0.1980
Epoch 3/5
500/500 [==============================] - 253s 507ms/step - loss: 3.4262 - acc: 0.1805 - val_loss: 3.2647 - val_acc: 0.2182
Epoch 4/5
500/500 [==============================] - 254s 507ms/step - loss: 3.3204 - acc: 0.1989 - val_loss: 3.0888 - val_acc: 0.2532
Epoch 5/5
500/500 [==============================] - 251s 502ms/step - loss: 3.2215 - acc: 0.2168 - val_loss: 3.1416 - val_acc: 0.2441


In [0]:
# making predictions
prediction = model.predict_classes(test)

In [0]:
# creating submission file
samp_csv['category'] = prediction
samp_csv.to_csv('sample_cnn2.csv', header=True, index=False)